<a href="https://colab.research.google.com/github/KIHOON71/pytorch_tutorial/blob/main/pytorch_tutorial_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#내장 데이터 셋에서 훈련 데이터 다운로드
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform = ToTensor()
)

#테스트 셋 다운로드
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()

)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size = 64

#데이터로더 만들기
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f'Shape of X [N, C, H, W] : {X.shape}')
  print(f'Shape of y : {y.shape}, {y.dtype}')
  break

Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]), torch.int64


##모델 만들기

In [ ]:
#Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f'using {device} device')

#모델 정의
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


##Optimizing

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #예측 에러 값
    pred = model(X)
    loss = loss_fn(pred, y)

    #역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')



In [ ]:

# def train(dataloader, model, loss_fn, optimizer):
#     size = len(dataloader.dataset)
#     model.train()
#     for batch, (X, y) in enumerate(dataloader):
#         X, y = X.to(device), y.to(device)

#         # Compute prediction error
#         pred = model(X)
#         loss = loss_fn(pred, y)

#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if batch % 100 == 0:
#             loss, current = loss.item(), batch * len(X)
#             print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f'Test Error : \n Accuracy : {(100*correct):>0.1f} %, AVG loss : {test_loss:>8f}\n')

##모델 훈련

In [ ]:
epochs = 5
for t in range(epochs):
  print(f'Epoch {t+1}\n---------------------------------------')
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!!")

Epoch 1
---------------------------------------
loss: 2.282791 [    0/60000]
loss: 2.275510 [ 6400/60000]
loss: 2.251966 [12800/60000]
loss: 2.254692 [19200/60000]
loss: 2.232157 [25600/60000]
loss: 2.189428 [32000/60000]
loss: 2.215044 [38400/60000]
loss: 2.166728 [44800/60000]
loss: 2.166518 [51200/60000]
loss: 2.129053 [57600/60000]
Test Error : 
 Accuracy : 34.4 %, AVG loss : 2.120656

Epoch 2
---------------------------------------
loss: 2.131435 [    0/60000]
loss: 2.119152 [ 6400/60000]
loss: 2.057035 [12800/60000]
loss: 2.077377 [19200/60000]
loss: 2.012804 [25600/60000]
loss: 1.946243 [32000/60000]
loss: 1.985487 [38400/60000]
loss: 1.895912 [44800/60000]
loss: 1.906127 [51200/60000]
loss: 1.822120 [57600/60000]
Test Error : 
 Accuracy : 56.7 %, AVG loss : 1.822868

Epoch 3
---------------------------------------
loss: 1.863826 [    0/60000]
loss: 1.823695 [ 6400/60000]
loss: 1.709510 [12800/60000]
loss: 1.746679 [19200/60000]
loss: 1.632373 [25600/60000]
loss: 1.596953 [32000

##Saving Model

In [31]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/pytorch_tutorial/model_tutorial.pth")
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


##Loading Models

In [32]:
model = NeuralNetwork()
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/pytorch_tutorial/model_tutorial.pth'))

<All keys matched successfully>

In [33]:
classes = [
           'T-shirt/top',
           'Trouser',
           'Pullover',
           'Dress',
           'Coat',
           'Sandal',
           'Shirt',
           'Sneaker',
           'Bag',
           'Ankle boot'
]

model.eval()
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: {predicted}, Actual : {actual}')


Predicted: Dress, Actual : Ankle boot
